In [54]:
import pyrealsense2.pyrealsense2 as rs
import numpy as np
import os
import cv2
from matplotlib import pyplot as plt
from fpdf import FPDF
import unicodedata

inputdirectory = "framesets"
outputdirectory = "faces"


def unicode_normalize(s):
    return unicodedata.normalize('NFKD', s).encode('ascii', 'ignore')

In [55]:
pdf = FPDF()
for filename in os.listdir(inputdirectory):
    print("Analyzing: {}".format(filename))
    # Configure depth and color streams
    pipeline = rs.pipeline()
    config = rs.config()
    colorizer = rs.colorizer()

    rs.config.enable_device_from_file(config, os.path.join(inputdirectory, filename))
    config.enable_stream(rs.stream.depth, rs.format.z16, 30)
    config.enable_stream(rs.stream.color, rs.format.bgr8, 30)

    # Start the pipeline
    profile = pipeline.start(config)

    # Extract depth scale
    depth_scale = profile.get_device().first_depth_sensor().get_depth_scale()

    # Initialize the Haar-cascade classifier
    face_cascade = cv2.CascadeClassifier();
    face_cascade.load("ext/haarcascade_frontalface_default.xml");

    # Wait for a coherent pair of frames: depth and color
    frames = pipeline.wait_for_frames()
    depth_frame = frames.get_depth_frame()
    depth_color_frame = rs.colorizer().colorize(depth_frame)
    color_frame = frames.get_color_frame()

    # Convert images to numpy arrays
    depth_image = np.asanyarray(depth_frame.get_data())
    depth_colormap = np.asanyarray(depth_color_frame.get_data())
    color_image = np.asanyarray(color_frame.get_data())
    gray_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)

    depth_colormap_dim = depth_colormap.shape
    color_colormap_dim = color_image.shape

    # Apply Haar-cascade classifier
    faces = face_cascade.detectMultiScale(gray_image)

    # Extract a copy of the original images to draw onto
    color_image_display = color_image.copy()
    depth_colormap_display = depth_colormap.copy()

    if len(faces) > 1:
        print("Warning! Multiple Faces Detected!")
    elif len(faces) < 1:
        print("Warning! No Faces Detected!")

    for i, (x,y,w,h) in enumerate(faces):
        extracted_face_color = color_image[y-30:y+h+30,x-30:x+w+30]
        extracted_face_color = np.rot90(extracted_face_color)
        cv2.imwrite(os.path.join(outputdirectory, filename + "_" + str(i) + ".png"), extracted_face_color)
        #outputHeight = 297-25.4*2
        outputWidth = 210-31.7*2 # = h+30
        outputHeight = ((297-25.4*2)/(h+60))*(210-31.7*2)
        
        pdf.add_page()
        pdf.image(os.path.join(outputdirectory, filename + "_" + str(i) + ".png"),0+31.7,0+25.4,outputWidth,outputHeight)

    cv2.destroyAllWindows()
    pipeline.stop()

# Save PDF
pdf.output("faces/extracted_faces.pdf", "F")

Analyzing: Classroom_10_Erik.bag
Analyzing: Classroom_11_Erwin.bag
Analyzing: Classroom_12_Tim.bag
Analyzing: Classroom_13_Tom.bag
Analyzing: Classroom_14_Johnny.bag
Analyzing: Classroom_15_Dennis.bag
Warning! Multiple Faces Detected!
Analyzing: Classroom_16_Dennis.bag
Analyzing: Classroom_17_Lisanne.bag
Analyzing: Classroom_18_Bryan.bag
Analyzing: Classroom_19_Eline.bag
Analyzing: Classroom_1_Sander.bag
Analyzing: Classroom_20_Rick.bag
Analyzing: Classroom_21_Arthur.bag
Analyzing: Classroom_2_Oscar.bag
Analyzing: Classroom_3_Kristo.bag
Analyzing: Classroom_4_Arthur.bag
Analyzing: Classroom_5_Dylan.bag
Warning! Multiple Faces Detected!
Analyzing: Classroom_6_Jacintha.bag
Analyzing: Classroom_7_Julia.bag
Analyzing: Classroom_8_Julia.bag
Analyzing: Classroom_9_Roald.bag
Analyzing: Test_1_Kasper.bag
Analyzing: Test_2_Kasper.bag
Warning! No Faces Detected!


''